<a href="https://colab.research.google.com/github/JozefSL/pyNotes/blob/main/EIA/weeklyXlsCrudeStocksAndChanges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go


In [2]:
file = r"https://ir.eia.gov/wpsr/psw01.xls"
data = pd.read_excel(file, sheet_name='Data 1', skiprows=1634, usecols=[0,1,2,4,10,19]) #index_col=0, , nrows=numRows)
data.columns = ['Week', 'COwSPR','CO' ,'Gasoline','Distillate','Petroleum']
#data.head(4)


In [3]:
data.tail(3)

,Week,COwSPR,CO,Gasoline,Distillate,Petroleum
558,2024-09-20,794935,413042,220083,122921,1267986
559,2024-09-27,799484,416931,221202,121637,1267077
560,2024-10-04,805671,422741,214898,118513,1258981


In [4]:
df = data[['Week','CO']].copy()
df.columns = ['Period', 'value']
df.tail(3)

,Period,value
558,2024-09-20,413042
559,2024-09-27,416931
560,2024-10-04,422741


In [5]:
df['Period'] = df['Period'].apply(pd.to_datetime)
df.value = (df.value.astype('Int64')/1000).round(1) #.astype('Int64')
df['year'] = df['Period'].dt.year
df['week'] = df['Period'].dt.isocalendar().week
df['change'] = df['value'] - df['value'].shift(1)
df.tail(3)

,Period,value,year,week,change
558,2024-09-20,413.0,2024,38,-4.5
559,2024-09-27,416.9,2024,39,3.9
560,2024-10-04,422.7,2024,40,5.8


In [6]:
ty = df.year.iloc[-1]
ly = ty-1

In [7]:
dfA = df.pivot(index='week', columns='year', values='value').round(1) #.astype('Int64') #.reset_index(drop='true')
dfA = dfA.drop(dfA.index[-1]).reset_index()  #eliminate the add years with 53 weeks

In [8]:
dfAA = pd.DataFrame(dfA.iloc[:, -6:-1].mean(axis=1))
dfAA.columns = ['5yAvg']
dfAA['5yMin'] = dfA.iloc[:, -6:-1].min(axis=1)
dfAA['5yMaxA'] = dfA.iloc[:, -6:-1].max(axis=1)
dfAA['5yMax'] = dfAA['5yMaxA']-dfAA['5yMin']
dfAA['10yAvg'] = dfA.iloc[:, -11:-1].mean(axis=1)
dfAA['10yMin'] = dfA.iloc[:, -11:-1].min(axis=1)
dfAA['10yMax'] = dfA.iloc[:, -11:-1].max(axis=1)
dfAA['week'] = dfA['week']

In [9]:
dfA['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfA['week'].astype(str)+'5',format='%Y%W%w')
dfAA['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAA['week'].astype(str)+'5',format='%Y%W%w')

In [10]:
dfAA.head(3)

,5yAvg,5yMin,5yMaxA,5yMax,10yAvg,10yMin,10yMax,week,weekDate
0,441.18,413.3,482.2,68.9,434.277778,348.8,483.1,1,2024-01-05
1,442.8,413.8,486.6,72.8,424.07,319.1,486.6,2,2024-01-12
2,442.9,416.2,476.7,60.5,426.24,320.1,488.3,3,2024-01-19


In [11]:
# Create the area chart
figCO = px.area(dfAA, x=dfAA.weekDate, y=['5yMin', '5yMax'], range_y=[400, 550], #range_x=[dfAA.index.min()-.2, dfAA.index.max()+.2],
              title='Stacked Area Chart with Line Chart').update_traces(fillcolor='rgba(128, 128, 128, 0.2)')

figCO.update_traces(line_color='rgba(0,0,0,0)',showlegend=False)

figCO.add_scatter(x=dfAA.weekDate, y=dfAA['5yAvg'], mode='lines', name='5yAvg', line=dict(color='darkgrey', width=2))
figCO.add_scatter(x=dfA.weekDate, y=dfA[ly], mode='lines', name=str(ly),line=dict(color='blue', width=2))
figCO.add_scatter(x=dfA.weekDate, y=dfA[ty], mode='lines+markers', name=str(ty),line=dict(color='red', width=3), marker=dict(color='red'))

# Customize the chart
figCO.update_layout(
    autosize=False,
    width=1200,
    height=800,
    title={
        'text': '<b>Total Crude Oil Stocks Excluding SPR</b>',
        'font': {'size': 28, 'family': 'Arial', 'color': 'black'}
        },
    xaxis_title='Week',
    yaxis_title='Crude Oil Stocks (MMbbl)'

)


figCO.update_traces(fillcolor='rgba(0,0,0,0)', selector=dict(name='5yMin'))


figCO.add_scatter(x = [dfA['weekDate'][dfA[ty].isna().idxmax()-1]], y = [dfA[ty][dfA[ty].isna().idxmax()-1]],
                     mode = 'markers + text',
                     marker = {'color':'red', 'size':14},
                     showlegend = False,
                     text = [dfA[ty][dfA[ty].isna().idxmax()-1]],
                     textfont=dict(color='red', size=16),
                     textposition='top right') # top, bottom, middle


figCO.show()

In [12]:
dfAc = df.pivot(index='week', columns='year', values='change').round(1) #.astype('Int64') #.reset_index(drop='true')
dfAc = dfAc.drop(dfAc.index[-1]).reset_index()  #eliminate the add years with 53 weeks
dfAc.tail(3)

year,week,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
49,50,-0.8,4.8,2.2,-6.5,-0.5,-1.1,-3.1,-4.7,-5.9,2.9,<NA>
50,51,7.2,-5.9,0.7,-4.6,-0.1,-5.4,-0.6,-3.6,0.8,-7.1,<NA>
51,52,-1.7,2.6,-7.1,-7.4,0.0,-11.5,-6.0,-2.1,1.6,-5.5,<NA>


In [13]:
dfAAc = pd.DataFrame(dfAc.iloc[:, -6:-1].mean(axis=1))
dfAAc.columns = ['5yAvg']
dfAAc['5yMin'] = dfAc.iloc[:, -6:-1].min(axis=1) # Actual minimum, also used to calculate stack area bottom
dfAAc['5yMaxA'] = dfAc.iloc[:, -6:-1].max(axis=1) # Actual maximum, also used to calculate stack area top
dfAAc['5yMax'] = dfAAc['5yMaxA']-dfAAc['5yMin']
dfAAc['week'] = dfAc['week']
dfAAc.head(3)

,5yAvg,5yMin,5yMaxA,5yMax,week
0,2.12,-4.6,19.0,23.6,1
1,1.62,-2.6,8.4,11.0,2
2,0.1,-9.9,7.9,17.8,3


In [14]:
dfAc['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAc['week'].astype(str)+'5',format='%Y%W%w')
dfAAc['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAAc['week'].astype(str)+'5',format='%Y%W%w')

In [15]:
categories = dfAAc['weekDate']
lower_bounds = dfAAc['5yMin']
upper_bounds = dfAAc['5yMaxA']

In [16]:
figCOc = go.Figure()

for i in range(len(categories)):
    figCOc.add_trace(go.Bar(
        x=[categories[i]],
        y=[upper_bounds[i] - lower_bounds[i]],
        base=[lower_bounds[i]],
        marker=dict(color='lightgrey'),
        showlegend=False
        #color_discrete_sequence=['blue']
        #name=str(categories[i])
    ))

figCOc.add_scatter(x=dfAAc.weekDate, y=dfAAc['5yAvg'], mode='markers', name='5yAvg', line=dict(color='grey', width=2))
figCOc.add_scatter(x=dfAc.weekDate, y=dfAc[ly], mode='markers', name=str(ly),line=dict(color='blue', width=2))
figCOc.add_scatter(x=dfAc.weekDate, y=dfAc[ty], mode='lines+markers', name=str(ty),line=dict(color='red', width=3), marker=dict(color='red'))

figCOc.add_scatter(x = [dfAc['weekDate'][dfAc[ty].isna().idxmax()-1]], y = [dfAc[ty][dfAc[ty].isna().idxmax()-1]],
                     mode = 'markers + text',
                     marker = {'color':'red', 'size':14},
                     showlegend = False,
                     text = [dfAc[ty][dfAc[ty].isna().idxmax()-1]],
                     textfont=dict(color='red', size=16),
                     textposition='top center') # top left, bottom right, middle left, bottom righ, top center, auto center

figCOc.update_layout(autosize=False,width=1200, height=700,
    title={
        'text': '<b>Crude Oil Weekly Changes</b>',
        'font': {'size': 28, 'family': 'Arial', 'color': 'black'}
        },
    #xaxis_title='Week',
    yaxis_title='Crude Oil Weekly Change (MMbbl)')

# Add source link as an annotation
figCOc.add_annotation(
    text="Source: EIA     https://www.eia.gov/petroleum/supply/weekly",
    xref="paper", yref="paper",
    x=0.95, y=0.05,
    showarrow=False,
    font=dict(size=13)
)

figCOc.show()

In [17]:
dfg = data[['Week','Gasoline']].copy()
dfg.columns = ['Period', 'value']
dfg.tail(3)

,Period,value
558,2024-09-20,220083
559,2024-09-27,221202
560,2024-10-04,214898


In [18]:
dfg['Period'] = dfg['Period'].apply(pd.to_datetime)
dfg.value = (dfg.value.astype('Int64')/1000).round(1) #.astype('Int64')
dfg['year'] = dfg['Period'].dt.year
dfg['week'] = dfg['Period'].dt.isocalendar().week
dfg['change'] = dfg['value'] - dfg['value'].shift(1)
dfg.tail(3)

,Period,value,year,week,change
558,2024-09-20,220.1,2024,38,-1.5
559,2024-09-27,221.2,2024,39,1.1
560,2024-10-04,214.9,2024,40,-6.3


In [19]:
dfAg = dfg.pivot(index='week', columns='year', values='value').round(1) #.astype('Int64') #.reset_index(drop='true')
dfAg = dfAg.drop(dfAg.index[-1]).reset_index()  #eliminate the add years with 53 weeks

In [20]:
dfAAg = pd.DataFrame(dfAg.iloc[:, -6:-1].mean(axis=1))
dfAAg.columns = ['5yAvg']
dfAAg['5yMin'] = dfAg.iloc[:, -6:-1].min(axis=1)
dfAAg['5yMaxA'] = dfAg.iloc[:, -6:-1].max(axis=1)
dfAAg['5yMax'] = dfAAg['5yMaxA']-dfAAg['5yMin']
dfAAg['week'] = dfAg['week']

In [21]:
dfAg['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAg['week'].astype(str)+'5',format='%Y%W%w')
dfAAg['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAAg['week'].astype(str)+'5',format='%Y%W%w')

In [22]:
dfAAg.head(3)

,5yAvg,5yMin,5yMaxA,5yMax,week,weekDate
0,242.54,226.8,251.6,24.8,1,2024-01-05
1,247.2,230.3,258.3,28.0,2,2024-01-12
2,249.44,232.0,260.0,28.0,3,2024-01-19


In [23]:
# Create the area chart
figG = px.area(dfAAg, x=dfAAg.weekDate, y=['5yMin', '5yMax'], range_y=[200, 270], #range_x=[dfAA.index.min()-.2, dfAA.index.max()+.2],
              title='Stacked Area Chart with Line Chart').update_traces(fillcolor='rgba(128, 128, 128, 0.2)')

figG.update_traces(line_color='rgba(0,0,0,0)',showlegend=False)

figG.add_scatter(x=dfAAg.weekDate, y=dfAAg['5yAvg'], mode='lines', name='5yAvg', line=dict(color='darkgrey', width=2))
figG.add_scatter(x=dfAg.weekDate, y=dfAg[ly], mode='lines', name=str(ly),line=dict(color='blue', width=2))
figG.add_scatter(x=dfAg.weekDate, y=dfAg[ty], mode='lines+markers', name=str(ty),line=dict(color='red', width=3), marker=dict(color='red'))

# Customize the chart
figG.update_layout(
    autosize=False,
    width=1200,
    height=800,
    title={
        'text': '<b>Total Gasoline Stocks</b>',
        'font': {'size': 28, 'family': 'Arial', 'color': 'black'}
        },
    xaxis_title='Week',
    yaxis_title='Total Gasoline Stocks (MMbbl)'

)


figG.update_traces(fillcolor='rgba(0,0,0,0)', selector=dict(name='5yMin'))


figG.add_scatter(x = [dfAg['weekDate'][dfAg[ty].isna().idxmax()-1]], y = [dfAg[ty][dfAg[ty].isna().idxmax()-1]],
                     mode = 'markers + text',
                     marker = {'color':'red', 'size':14},
                     showlegend = False,
                     text = [dfAg[ty][dfAg[ty].isna().idxmax()-1]],
                     textfont=dict(color='red', size=16),
                     textposition='bottom right') # top, bottom, middle


figG.show()

In [24]:
dfAgc = dfg.pivot(index='week', columns='year', values='change').round(1) #.astype('Int64') #.reset_index(drop='true')
dfAgc = dfAgc.drop(dfAgc.index[-1]).reset_index()  #eliminate the add years with 53 weeks
dfAgc.tail(3)

year,week,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
49,50,5.2,1.7,-1.3,1.3,1.8,2.5,1.0,5.5,2.5,2.7,<NA>
50,51,4.1,1.1,-1.6,0.6,3.0,2.0,-1.1,-1.4,-3.1,-0.6,<NA>
51,52,2.9,0.9,8.3,4.8,6.9,3.2,-1.2,10.1,-0.3,10.9,<NA>


In [25]:
dfAAgc = pd.DataFrame(dfAgc.iloc[:, -6:-1].mean(axis=1))
dfAAgc.columns = ['5yAvg']
dfAAgc['5yMin'] = dfAgc.iloc[:, -6:-1].min(axis=1) # Actual minimum, also used to calculate stack area bottom
dfAAgc['5yMaxA'] = dfAgc.iloc[:, -6:-1].max(axis=1) # Actual maximum, also used to calculate stack area top
dfAAgc['5yMax'] = dfAAgc['5yMaxA']-dfAAgc['5yMin']
dfAAgc['week'] = dfAgc['week']
dfAAgc.head(3)

,5yAvg,5yMin,5yMaxA,5yMax,week
0,6.72,4.1,9.1,5.0,1
1,4.66,-0.3,7.5,7.8,2
2,2.24,1.3,4.0,2.7,3


In [26]:
dfAgc['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAgc['week'].astype(str)+'5',format='%Y%W%w')
dfAAgc['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAAgc['week'].astype(str)+'5',format='%Y%W%w')

In [27]:
categories = dfAAgc['weekDate']
lower_bounds = dfAAgc['5yMin']
upper_bounds = dfAAgc['5yMaxA']

In [28]:
figGc = go.Figure()

for i in range(len(categories)):
    figGc.add_trace(go.Bar(
        x=[categories[i]],
        y=[upper_bounds[i] - lower_bounds[i]],
        base=[lower_bounds[i]],
        marker=dict(color='lightgrey'),
        showlegend=False
        #color_discrete_sequence=['blue']
        #name=str(categories[i])
    ))

figGc.add_scatter(x=dfAAgc.weekDate, y=dfAAgc['5yAvg'], mode='markers', name='5yAvg', line=dict(color='grey', width=2))
figGc.add_scatter(x=dfAgc.weekDate, y=dfAgc[ly], mode='markers', name=str(ly),line=dict(color='blue', width=2))
figGc.add_scatter(x=dfAgc.weekDate, y=dfAgc[ty], mode='lines+markers', name=str(ty),line=dict(color='red', width=3), marker=dict(color='red'))

figGc.add_scatter(x = [dfAgc['weekDate'][dfAgc[ty].isna().idxmax()-1]], y = [dfAgc[ty][dfAgc[ty].isna().idxmax()-1]],
                     mode = 'markers + text',
                     marker = {'color':'red', 'size':14},
                     showlegend = False,
                     text = [dfAgc[ty][dfAgc[ty].isna().idxmax()-1]],
                     textfont=dict(color='red', size=16),
                     textposition='bottom center') # top left, bottom right, middle left, bottom righ, top center, auto center

figGc.update_layout(autosize=False,width=1200, height=700,
    title={
        'text': '<b>Total Gasoline Stocks Weekly Changes</b>',
        'font': {'size': 28, 'family': 'Arial', 'color': 'black'}
        },
    #xaxis_title='Week',
    yaxis_title='Total Gasoline Stocks Weekly Change (MMbbl)')

# Add source link as an annotation
figGc.add_annotation(
    text="Source: EIA     https://www.eia.gov/petroleum/supply/weekly",
    xref="paper", yref="paper",
    x=0.95, y=0.05,
    showarrow=False,
    font=dict(size=13)
)

figGc.show()

In [29]:
dfd = data[['Week','Distillate']].copy()
dfd.columns = ['Period', 'value']
dfd.tail(3)

,Period,value
558,2024-09-20,122921
559,2024-09-27,121637
560,2024-10-04,118513


In [30]:
dfd['Period'] = dfd['Period'].apply(pd.to_datetime)
dfd.value = (dfd.value.astype('Int64')/1000).round(1) #.astype('Int64')
dfd['year'] = dfd['Period'].dt.year
dfd['week'] = dfd['Period'].dt.isocalendar().week
dfd['change'] = dfd['value'] - dfd['value'].shift(1)
dfd.tail(3)

,Period,value,year,week,change
558,2024-09-20,122.9,2024,38,-2.2
559,2024-09-27,121.6,2024,39,-1.3
560,2024-10-04,118.5,2024,40,-3.1


In [31]:
dfAd = dfd.pivot(index='week', columns='year', values='value').round(1) #.astype('Int64') #.reset_index(drop='true')
dfAd = dfAd.drop(dfAd.index[-1]).reset_index()  #eliminate the add years with 53 weeks

In [32]:
dfAAd = pd.DataFrame(dfAd.iloc[:, -6:-1].mean(axis=1))
dfAAd.columns = ['5yAvg']
dfAAd['5yMin'] = dfAd.iloc[:, -6:-1].min(axis=1)
dfAAd['5yMaxA'] = dfAd.iloc[:, -6:-1].max(axis=1)
dfAAd['5yMax'] = dfAAd['5yMaxA']-dfAAd['5yMin']
dfAAd['week'] = dfAd['week']

In [33]:
dfAd['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAd['week'].astype(str)+'5',format='%Y%W%w')
dfAAd['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAAd['week'].astype(str)+'5',format='%Y%W%w')

In [34]:
dfAAd.head(3)

,5yAvg,5yMin,5yMaxA,5yMax,week,weekDate
0,137.86,117.7,163.2,45.5,1,2024-01-05
1,139.54,115.8,163.7,47.9,2,2024-01-12
2,138.34,115.3,162.8,47.5,3,2024-01-19


In [35]:
# Create the area chart
figD = px.area(dfAAd, x=dfAAd.weekDate, y=['5yMin', '5yMax'], range_y=[100, 180],  #range_x=[dfAAd.index.min()-.2, dfAA.index.max()+.2],
              title='Stacked Area Chart with Line Chart').update_traces(fillcolor='rgba(128, 128, 128, 0.2)')

figD.update_traces(line_color='rgba(0,0,0,0)',showlegend=False)

figD.add_scatter(x=dfAAd.weekDate, y=dfAAd['5yAvg'], mode='lines', name='5yAvg', line=dict(color='darkgrey', width=2))
figD.add_scatter(x=dfAd.weekDate, y=dfAd[ly], mode='lines', name=str(ly),line=dict(color='blue', width=2))
figD.add_scatter(x=dfAd.weekDate, y=dfAd[ty], mode='lines+markers', name=str(ty),line=dict(color='red', width=3), marker=dict(color='red'))

# Customize the chart
figD.update_layout(
    autosize=False,
    width=1210,
    height=800,
    title={
        'text': '<b>Distillates Stocks</b>',
        'font': {'size': 28, 'family': 'Arial', 'color': 'black'}
        },
    xaxis_title='Week',
    yaxis_title='Distillates Stocks (MMbbl)'

)


figD.update_traces(fillcolor='rgba(0,0,0,0)', selector=dict(name='5yMin'))


figD.add_scatter(x = [dfAd['weekDate'][dfAd[ty].isna().idxmax()-1]], y = [dfAd[ty][dfAd[ty].isna().idxmax()-1]],
                     mode = 'markers + text',
                     marker = {'color':'red', 'size':14},
                     showlegend = False,
                     text = [dfA[ty][dfA[ty].isna().idxmax()-1]],
                     textfont=dict(color='red', size=16),
                     textposition='bottom right') # top, bottom, middle


figD.show()

In [36]:
dfAdc = dfd.pivot(index='week', columns='year', values='change').round(1) #.astype('Int64') #.reset_index(drop='true')
dfAdc = dfAdc.drop(dfAdc.index[-1]).reset_index()  #eliminate the add years with 53 weeks
dfAdc.tail(3)

year,week,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
49,50,-0.3,2.6,-2.4,0.7,-4.2,1.5,0.2,0.4,-0.3,1.5,<NA>
50,51,2.3,-0.7,-1.9,1.1,0.0,-0.2,-2.4,-1.8,0.3,0.8,<NA>
51,52,1.9,1.8,10.1,8.9,9.5,8.8,3.1,4.4,-1.4,10.1,<NA>


In [37]:
dfAAdc = pd.DataFrame(dfAdc.iloc[:, -6:-1].mean(axis=1))
dfAAdc.columns = ['5yAvg']
dfAAdc['5yMin'] = dfAdc.iloc[:, -6:-1].min(axis=1) # Actual minimum, also used to calculate stack area bottom
dfAAdc['5yMaxA'] = dfAdc.iloc[:, -6:-1].max(axis=1) # Actual maximum, also used to calculate stack area top
dfAAdc['5yMax'] = dfAAdc['5yMaxA']-dfAAdc['5yMin']
dfAAdc['week'] = dfAdc['week']
dfAAdc.head(3)

,5yAvg,5yMin,5yMaxA,5yMax,week
0,4.44,-1.1,10.6,11.7,1
1,1.68,-1.9,8.2,10.1,2
2,-1.2,-2.8,-0.5,2.3,3


In [38]:
dfAdc['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAdc['week'].astype(str)+'5',format='%Y%W%w')
dfAAdc['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAAdc['week'].astype(str)+'5',format='%Y%W%w')

In [39]:
categories = dfAAdc['weekDate']
lower_bounds = dfAAdc['5yMin']
upper_bounds = dfAAdc['5yMaxA']

In [40]:
figDc = go.Figure()

for i in range(len(categories)):
    figDc.add_trace(go.Bar(
        x=[categories[i]],
        y=[upper_bounds[i] - lower_bounds[i]],
        base=[lower_bounds[i]],
        marker=dict(color='lightgrey'),
        showlegend=False
        #color_discrete_sequence=['blue']
        #name=str(categories[i])
    ))

figDc.add_scatter(x=dfAAdc.weekDate, y=dfAAdc['5yAvg'], mode='markers', name='5yAvg', line=dict(color='grey', width=2))
figDc.add_scatter(x=dfAdc.weekDate, y=dfAdc[ly], mode='markers', name=str(ly),line=dict(color='blue', width=2))
figDc.add_scatter(x=dfAdc.weekDate, y=dfAdc[ty], mode='lines+markers', name=str(ty),line=dict(color='red', width=3), marker=dict(color='red'))

figDc.add_scatter(x = [dfAdc['weekDate'][dfAdc[ty].isna().idxmax()-1]], y = [dfAdc[ty][dfAdc[ty].isna().idxmax()-1]],
                     mode = 'markers + text',
                     marker = {'color':'red', 'size':14},
                     showlegend = False,
                     text = [dfAdc[ty][dfAdc[ty].isna().idxmax()-1]],
                     textfont=dict(color='red', size=16),
                     textposition='bottom center') # top left, bottom right, middle left, bottom righ, top center, auto center

figDc.update_layout(autosize=False,width=1200, height=700,
    title={
        'text': '<b>Distillates Stocks Weekly Changes</b>',
        'font': {'size': 28, 'family': 'Arial', 'color': 'black'}
        },
    #xaxis_title='Week',
    yaxis_title='Distillatese Stocks Weekly Change (MMbbl)')

# Add source link as an annotation
figDc.add_annotation(
    text="Source: EIA     https://www.eia.gov/petroleum/supply/weekly",
    xref="paper", yref="paper",
    x=0.95, y=0.05,
    showarrow=False,
    font=dict(size=13)
)

figDc.show()

In [41]:
dfp = data[['Week','Petroleum']].copy()
dfp.columns = ['Period', 'value']
dfp.tail(3)

,Period,value
558,2024-09-20,1267986
559,2024-09-27,1267077
560,2024-10-04,1258981


In [42]:
dfp['Period'] = dfp['Period'].apply(pd.to_datetime)
dfp.value = (dfp.value.astype('Int64')/1000).round(1) #.astype('Int64')
dfp['year'] = dfp['Period'].dt.year
dfp['week'] = dfp['Period'].dt.isocalendar().week
dfp['change'] = dfp['value'] - dfp['value'].shift(1)
dfp.tail(3)

,Period,value,year,week,change
558,2024-09-20,1268.0,2024,38,-14.6
559,2024-09-27,1267.1,2024,39,-0.9
560,2024-10-04,1259.0,2024,40,-8.1


In [43]:
dfAp = dfp.pivot(index='week', columns='year', values='value').round(1) #.astype('Int64') #.reset_index(drop='true')
dfAp = dfAp.drop(dfAp.index[-1]).reset_index()  #eliminate the add years with 53 weeks

In [44]:
dfAAp = pd.DataFrame(dfAp.iloc[:, -6:-1].mean(axis=1))
dfAAp.columns = ['5yAvg']
dfAAp['5yMin'] = dfAp.iloc[:, -6:-1].min(axis=1)
dfAAp['5yMaxA'] = dfAp.iloc[:, -6:-1].max(axis=1)
dfAAp['5yMax'] = dfAAp['5yMaxA']-dfAAp['5yMin']
dfAAp['10yAvg'] = dfAp.iloc[:, -11:-1].mean(axis=1)
dfAAp['10yMin'] = dfAp.iloc[:, -11:-1].min(axis=1)
dfAAp['10yMax'] = dfAp.iloc[:, -11:-1].max(axis=1)
dfAAp['week'] = dfAp['week']

In [45]:
dfAp['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAp['week'].astype(str)+'5',format='%Y%W%w')
dfAAp['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAAp['week'].astype(str)+'5',format='%Y%W%w')

In [46]:
figP = px.area(dfAAp, x=dfAAp.weekDate, y=['5yMin', '5yMax'], range_y=[1100, 1500], #range_x=[dfAA.index.min()-.2, dfAA.index.max()+.2],
              title='Stacked Area Chart with Line Chart').update_traces(fillcolor='rgba(128, 128, 128, 0.2)')

figP.update_traces(line_color='rgba(0,0,0,0)',showlegend=False)

figP.add_scatter(x=dfAAp.weekDate, y=dfAAp['5yAvg'], mode='lines', name='5yAvg', line=dict(color='darkgrey', width=2))
figP.add_scatter(x=dfAp.weekDate, y=dfAp[ly], mode='lines', name=str(ly),line=dict(color='blue', width=2))
figP.add_scatter(x=dfAp.weekDate, y=dfAp[ty], mode='lines+markers', name=str(ty),line=dict(color='red', width=3), marker=dict(color='red'))

# Customize the chart
figP.update_layout(
    autosize=False,
    width=1200,
    height=800,
    title={
        'text': '<b>Total Petroleum Stocks</b>',
        'font': {'size': 28, 'family': 'Arial', 'color': 'black'}
        },
    xaxis_title='Week',
    yaxis_title='Total Petroleum Stocks (MMbbl)'

)


figP.update_traces(fillcolor='rgba(0,0,0,0)', selector=dict(name='5yMin'))


figP.add_scatter(x = [dfAp['weekDate'][dfAp[ty].isna().idxmax()-1]], y = [dfAp[ty][dfAp[ty].isna().idxmax()-1]],
                     mode = 'markers + text',
                     marker = {'color':'red', 'size':14},
                     showlegend = False,
                     text = [dfAp[ty][dfAp[ty].isna().idxmax()-1]],
                     textfont=dict(color='red', size=16),
                     textposition='bottom right') # top, bottom, middle


figP.show()

In [47]:
dfApc = dfp.pivot(index='week', columns='year', values='change').round(1) #.astype('Int64') #.reset_index(drop='true')
dfApc = dfApc.drop(dfApc.index[-1]).reset_index()  #eliminate the add years with 53 weeks
dfApc.tail(3)

year,week,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
49,50,1.2,4.8,-11.9,-14.3,-10.6,-1.1,-6.3,-7.0,-11.6,2.7,<NA>
50,51,6.4,-5.2,-13.1,-8.9,-2.2,-10.4,-10.6,-18.9,-11.2,-17.8,<NA>
51,52,2.8,-0.1,6.3,1.3,14.4,-3.1,-15.0,10.2,-3.2,6.9,<NA>


In [48]:
dfAApc = pd.DataFrame(dfApc.iloc[:, -6:-1].mean(axis=1))
dfAApc.columns = ['5yAvg']
dfAApc['5yMin'] = dfApc.iloc[:, -6:-1].min(axis=1) # Actual minimum, also used to calculate stack area bottom
dfAApc['5yMaxA'] = dfApc.iloc[:, -6:-1].max(axis=1) # Actual maximum, also used to calculate stack area top
dfAApc['5yMax'] = dfAApc['5yMaxA']-dfAApc['5yMin']
dfAApc['week'] = dfApc['week']
dfAApc.head(3)

,5yAvg,5yMin,5yMaxA,5yMax,week
0,7.26,-9.4,22.4,31.8,1
1,3.88,-1.5,14.5,16.0,2
2,-1.42,-11.7,6.4,18.1,3


In [49]:
dfApc['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfApc['week'].astype(str)+'5',format='%Y%W%w')
dfAApc['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAApc['week'].astype(str)+'5',format='%Y%W%w')

In [50]:
categories = dfAApc['weekDate']
lower_bounds = dfAApc['5yMin']
upper_bounds = dfAApc['5yMaxA']

In [52]:
figPc = go.Figure()

for i in range(len(categories)):
    figPc.add_trace(go.Bar(
        x=[categories[i]],
        y=[upper_bounds[i] - lower_bounds[i]],
        base=[lower_bounds[i]],
        marker=dict(color='lightgrey'),
        showlegend=False
        #color_discrete_sequence=['blue']
        #name=str(categories[i])
    ))

figPc.add_scatter(x=dfAApc.weekDate, y=dfAApc['5yAvg'], mode='markers', name='5yAvg', line=dict(color='grey', width=2))
figPc.add_scatter(x=dfApc.weekDate, y=dfApc[ly], mode='markers', name=str(ly),line=dict(color='blue', width=2))
figPc.add_scatter(x=dfApc.weekDate, y=dfApc[ty], mode='lines+markers', name=str(ty),line=dict(color='red', width=3), marker=dict(color='red'))

figPc.add_scatter(x = [dfApc['weekDate'][dfApc[ty].isna().idxmax()-1]], y = [dfApc[ty][dfApc[ty].isna().idxmax()-1]],
                     mode = 'markers + text',
                     marker = {'color':'red', 'size':14},
                     showlegend = False,
                     text = [dfApc[ty][dfApc[ty].isna().idxmax()-1]],
                     textfont=dict(color='red', size=16),
                     textposition='bottom center') # top left, bottom right, middle left, bottom righ, top center, auto center

figPc.update_layout(autosize=False,width=1200, height=700,
    title={
        'text': '<b>Total Petroleum Stocks Weekly Changes</b>',
        'font': {'size': 28, 'family': 'Arial', 'color': 'black'}
        },
    #xaxis_title='Week',
    yaxis_title='Total Petroleum Stocks Weekly Change (MMbbl)')

# Add source link as an annotation
figPc.add_annotation(
    text="Source: EIA     https://www.eia.gov/petroleum/supply/weekly",
    xref="paper", yref="paper",
    x=0.95, y=0.05,
    showarrow=False,
    font=dict(size=13)
)

figPc.show()